### This will be the continuously tested version. Testing mostly done in Spyder, then the correct code is uploaded here.

In [1]:
import tkinter as tk
from tkinter import simpledialog, scrolledtext
from tkinter import messagebox #this is for pop up when illegal characters was input
import time
import csv
import os

def save_data(data, filepath):
    headers = ["ID", "TotalKeystrokes1", "TotalBackspaces1", "TimeElapsed1", "InputText1", "StressRating1", 
               "TotalKeystrokes2", "TotalBackspaces2", "TimeElapsed2", "InputText2", "StressRating2"]
    file_exists = os.path.isfile(filepath)
    with open(filepath, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)

def open_summary_window(data, root, round_index):
    summary_window = tk.Toplevel(root)
    summary_window.title(f"Summary of Your Input - Round {round_index}")
    summary_window.geometry("300x200")
    text_display = (f"ID: {data['ID']}\n"
                    f"Total Keystrokes: {data[f'TotalKeystrokes{round_index}']}\n"
                    f"Total Backspaces: {data[f'TotalBackspaces{round_index}']}\n"
                    f"Time Elapsed: {data[f'TimeElapsed{round_index}']} seconds\n"
                    f"Stress Rating: {data[f'StressRating{round_index}']}")
    summary_label = tk.Label(summary_window, text=text_display, justify="left", wraplength=280)
    summary_label.pack(pady=20)

    if round_index == "2":
        save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv")
        close_button = tk.Button(summary_window, text="Save and Exit", command=lambda: end_and_save(root))
        close_button.pack(side=tk.LEFT, padx=10, pady=5)
        return_button = tk.Button(summary_window, text="Return to Beginning", command=lambda: return_to_start(summary_window, root))
        return_button.pack(side=tk.RIGHT, padx=10, pady=5)
    else:
        next_button = tk.Button(summary_window, text="Next", command=lambda: [summary_window.destroy(), open_reply_window(root, data["ID"], "2", data)])
        next_button.pack(pady=5)

def open_reply_window(root, user_id, round_index, data):
    reply_window = tk.Toplevel(root)
    reply_window.title(f"Reply to Supervisor - Round {round_index}")
    reply_window.geometry("600x400")
    email_message = {
        "1": "你好,\n\n我注意到你之前请假，现在可能需要时间来重新调整状态。 "
             "然而，我也想提醒你，我们的项目进展需要你的及时参与和努力，所以需要你加紧完成。 "
             "就目前来看，你之前请假的理由不足以解释你现在仍未完成项目的情况。 "
             "我希望你能赶紧进入状态投入工作，确保我们组能够按时完成项目。\n\n"
             "有问题请及时与我沟通。\n--项目组组长",
        "2": "Hello,\n\nI noticed that you previously took leave and may now need time to readjust."
             "However, I also want to remind you that the progress of our project requires your timely"
             "participation and effort, so we need you to expedite your progress."
             "Based on the current situation, the reasons for your previous leave are insufficient"
             "to explain why you have not yet completed the project."
             "Please communicate with me promptly if you have any questions."
             "\n\nBest regards,\nYour Supervisor"
    }[round_index]
    message_label = tk.Label(reply_window, text=email_message, justify="left", wraplength=580)
    message_label.pack(pady=(10, 20))
    text_box = scrolledtext.ScrolledText(reply_window, width=50, height=10)
    text_box.pack()
    timer_label = tk.Label(reply_window, text="00:00", font=('Helvetica', 14))
    timer_label.pack(anchor='ne')
    start_time = time.time()

    def update_timer():
        if not getattr(reply_window, 'stop_timer', False):
            elapsed_time = time.time() - start_time
            minutes, seconds = divmod(int(elapsed_time), 60)
            timer_label.config(text=f"{minutes:02}:{seconds:02}")
            reply_window.after(1000, update_timer)
    update_timer()

    def handle_send(data):
        reply_window.stop_timer = True
        characters = len(text_box.get("1.0", "end-1c"))
        input_text = text_box.get("1.0", "end-1c")
        elapsed_time = time.time() - start_time
        data[f"TotalKeystrokes{round_index}"] = characters
        data[f"TotalBackspaces{round_index}"] = 0 # Update if tracking backspaces
        data[f"TimeElapsed{round_index}"]     = int(elapsed_time)
        data[f"InputText{round_index}"]       = input_text
        data[f"StressRating{round_index}"]    = 0 # Initialized for update in stress rating
        #save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv")  # Save data after sending
        reply_window.destroy()
        open_stress_window(data, root, round_index)
    send_button = tk.Button(reply_window, text="Send", command=lambda: handle_send(data))
    send_button.pack(pady=10)
    reply_window.protocol("WM_DELETE_WINDOW", lambda: handle_send(data))  # Handle window close as send

def end_and_save(root):
    root.quit()
    root.destroy()
    print("All windows closed and program terminated")

def open_stress_window(data, root, round_index):
    stress_window = tk.Toplevel(root)
    stress_window.title("Stress Level Rating")
    stress_window.geometry("400x200")

    instruction_text = "Rate the level of stress experienced (0-10, decimals allowed):"
    instruction_label = tk.Label(stress_window, text=instruction_text, wraplength=380)
    instruction_label.pack(pady=20)

    stress_entry = tk.Entry(stress_window)
    stress_entry.pack(pady=10)

    submit_button = tk.Button(stress_window, text="Submit", command=lambda: save_stress_and_proceed(data, stress_entry.get(), stress_window, root, round_index))
    submit_button.pack(pady=10)

def save_stress_and_proceed(data, stress_rating, stress_window, root, round_index):
    try:
        stress_rating = float(stress_rating)  # Ensure stress_rating is converted to float
        data[f'StressRating{round_index}'] = stress_rating  # Update the stress rating in data
        stress_window.destroy()  # Close the stress rating window
        #save_data(data, "/Users/ariel/Desktop/Spring24/525/GUIproject/GUIdata.csv")  # Save updated data
        open_summary_window(data, root, round_index)  # Open the summary window
    except ValueError:
        # Handle the case where the input is not a valid float
        instruction_label.config(text="Invalid input. Please enter a numeric value (0-10).")
    
def return_to_start(summary_window, root):
    summary_window.destroy()
    open_instructions_window(root)

def open_instructions_window(root):
    instructions_window = tk.Toplevel(root)
    instructions_window.title("Instructions")
    instructions_window.geometry("600x400")
    instruction_text = ("You have entered your ID. Please read the following instructions carefully:\n"
                        "1. You will read two emails sent by your supervisor.\n"
                        "2. Round 1 will be in Chinese and Round 2 in English.\n"
                        "3. Please respond ONLY in ENGLISH.\n"
                        "4. Please do not click SEND until you are satisfied with your response.\n"
                        "5. Please do not click SAVE AND EXIT until your experimenter says so.\n")
    instructions_label = tk.Label(instructions_window, text=instruction_text, justify="left", wraplength=580)
    instructions_label.pack(pady=20)
    
    # Keep the instructions window on top until a valid ID is entered
    instructions_window.grab_set()
    
    user_id = None
    while not user_id:
        user_id = simpledialog.askstring("Input", "Enter your ID number:", parent=instructions_window)
        if user_id:  # If a user ID was entered
            data = {}
            data.setdefault("ID", user_id)
            instructions_window.destroy()  # Destroy the window as a valid ID is entered
            open_reply_window(root, user_id, "1", data)
        else:
            messagebox.showwarning("ID Required", "You must enter an ID to proceed.", parent=instructions_window)

def main():
    root = tk.Tk()
    root.withdraw()
    open_instructions_window(root)
    root.mainloop()

if __name__ == "__main__":
    main()

2024-05-07 15:50:08.551 python[87237:10005413] Warning: Expected min height of view: (<NSButton: 0x7f98c57bfdc0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2024-05-07 15:50:12.941 python[87237:10005413] Warning: Expected min height of view: (<NSButton: 0x7f98c7011c20>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


All windows closed and program terminated
